In [1]:
import xlwings as xw
import pandas as pd
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']

In [2]:
import cx_Oracle
import os
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"

In [3]:
def DataWarehouse():
    try:
        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
        cx_Oracle.init_oracle_client(lib_dir=LOCATION)
        connection = cx_Oracle.connect(
            user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

        )
        cursor = connection.cursor()
        return cursor

    except:

        connection = cx_Oracle.connect(
            user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

        )
        cursor = connection.cursor()
        return cursor

In [4]:
cur = DataWarehouse()

In [59]:
dic_dm = dict(cur.execute('select * from DELIVERY_METHOD').fetchall())

In [66]:
dict(zip(list(dic_dm.values()),list(dic_dm.keys())))

{'일반배송': 0, '택배': 1, '무진동차량': 2, '직배송': 3, 'ELSE': 99}

['일반배송', '택배', '무진동차량', '직배송', 'ELSE']

In [ ]:
inv_map = {v: k for k, v in map.items()}

In [8]:
import pandas as pd

In [19]:
df_dm= pd.DataFrame(cur.execute('select * from DELIVERY_METHOD').fetchall())

In [56]:
dict_dm = dict(zip(list(df_dm[1]),list(df_dm[0])))

In [57]:
dict_dm['일반배송']

0

In [70]:
def get_tb_idx(tb_name=str, content=str):
    """
    DW의 table이름을, content에는 테이블의 content를 입력하면 tb상 key값을 반환한다.
    """
    cur = DataWarehouse()
    dic_dm = dict(cur.execute(f'select * from {tb_name}').fetchall())
    dic_dm = dict(zip(list(dic_dm.values()),list(dic_dm.keys())))
    return dic_dm[content]

In [107]:
df_test = pd.read_csv('../test.csv',encoding='utf-8',index_col=0)

In [97]:
df_test['0'][0] = None

In [108]:
df_test.index=df_test.index+1

In [110]:
df_test.columns

Index(['0'], dtype='object')

# 실시간 재고리스트 업데이트


In [131]:
ws_stocks = wb_cy.sheets['재고리스트']

In [132]:
last_row = ws_stocks.range("A1000000").end('up').row
last_col = ws_stocks.range("AAA1").end('left').column

In [133]:
datas_stocks = ws_stocks.range((2,1),(last_row,last_col)).options(numbers=int).value
col_names = ws_stocks.range((1,1),(1,last_col)).value

In [134]:
df_stock = pd.DataFrame(datas_stocks,columns=col_names)
df_stock 

,OU NAME,COUNTRY,PROD CENTRE,PROD GROUP,ARTICLE NUMBER,DESCRIPTION,PROD STATUS TYPE,LOT NUMBER,SERIAL NUMBER,SUBINVENTORY,LOCATION,QUANTITY,IN DATE,EXPIRY DATE,CURRENCY,LOT COST,LOT COST IN USD,LOT COST IN EUR
0,GEAS_OU_KRW_LS_KR,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,607 AKTA legacy parts,11000314,CU-950 KIT FOR AKTA,DstFGActv,None,None,KR_SERV01,None,1,2022-12-05,NaT,KRW,2291234,1817,1697
1,GEAS_OU_KRW_LS_KR,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,922 Bioprocess Downstream Parts - Standard,11000322,CU-950 MEMORY KIT,DstFGActv,None,None,KR_CSW,None,1,2023-01-05,NaT,KRW,175786,139,130
2,GEAS_OU_KRW_LS_KR,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,922 Bioprocess Downstream Parts - Standard,11000322,CU-950 MEMORY KIT,DstFGActv,None,None,KR_SERV01,None,1,2022-08-07,NaT,KRW,175786,139,130
3,GEAS_OU_KRW_LS_KR,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,607 AKTA legacy parts,11000735,INTERFERENCE FILTER 280 NM,DstFGActv,None,None,KR_SERV01,None,1,2022-12-12,NaT,KRW,420510,333,311
4,GEAS_OU_KRW_LS_KR,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,607 AKTA legacy parts,11000735,INTERFERENCE FILTER 280 NM,DstFGActv,None,None,KR_SERV01,None,1,2022-12-12,NaT,KRW,420510,333,311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,GEAS_OU_KRW_LS_KR,KRS,48 RESEARCH SCIENCE SERVICE PARTS,139 Computer Hardware,82000250,"Domestic purchasing computer set worth of 2,40...",DstFGActv,800673,800676,KR_LOCN02,None,1,2022-12-05,NaT,KRW,2500000,1982,1852
1661,GEAS_OU_KRW_LS_KR,KRS,48 RESEARCH SCIENCE SERVICE PARTS,139 Computer Hardware,82000250,"Domestic purchasing computer set worth of 2,40...",DstFGActv,800673,800677,KR_LOCN02,None,1,2022-12-05,NaT,KRW,2500000,1982,1852
1662,GEAS_OU_KRW_LS_KR,KRS,20 CRYO CHAIN MANAGEMENT,157 VIA Thaw,TBA_30001,VIA Thaw CB1000 Cryo-bag Thawing System,DstFGActv,None,TBA1118020,KR_DEMO,NULL.0.0.0.KR,1,2019-06-21,NaT,KRW,15036302,11924,11137
1663,GEAS_OU_KRW_LS_KR,KRS,20 CRYO CHAIN MANAGEMENT,156 VIA Freeze,VFD_30006,VIA Freeze Duo Controlled Rate Freezer,DstFGActv,None,VFD0920022,KR_DEMO,NULL.0.0.0.KR,1,2021-06-22,NaT,KRW,43671244,34631,32348


# BIN

In [113]:
ws_bin = wb_cy.sheets['Cytiva Inventory BIN']

In [114]:
last_row = ws_bin.range("A1000000").end('up').row
last_col = ws_bin.range("AAA1").end('left').column

In [115]:
datas_bin = ws_bin.range((2,1),(last_row,last_col)).options(numbers=int).value
col_names = ws_bin.range((1,1),(1,last_col)).options(numbers=int).value
df_bin = pd.DataFrame(datas_bin,columns=col_names)
df_bin

,ARTICLENUMBER,BIN(current),BIN(old),SUBINVENTORY
0,34-100390-002-9,"2C1804,2C1405",1Y0804,KR_SERV01
1,29128030,2B1801,1X0801,KR_SERV01
2,29089545,"BULK,2C1405",BULK,KR_SERV01
3,29088185,"2C1604,2C1603",1Y0604,KR_SERV01
4,29088182,"2C1602,BULK,2C1603",1Y0602,KR_SERV01
...,...,...,...,...
1032,18102926,2B1804,None,KR_SERV01
1033,19781401,2B1803,None,KR_SERV01
1034,29282854,2B1802,None,KR_SERV01
1035,29392877,2B1802,None,KR_SERV01


In [116]:
# PK 빈칸 없는지 확인
df_bin[~pd.notna(df_bin['ARTICLENUMBER'])]

,ARTICLENUMBER,BIN(current),BIN(old),SUBINVENTORY


In [117]:
# PK 중복확인
part_nms=pd.Series(df_bin['ARTICLENUMBER'])
len(part_nms.unique())

1037

In [118]:
df_bin

,ARTICLENUMBER,BIN(current),BIN(old),SUBINVENTORY
0,34-100390-002-9,"2C1804,2C1405",1Y0804,KR_SERV01
1,29128030,2B1801,1X0801,KR_SERV01
2,29089545,"BULK,2C1405",BULK,KR_SERV01
3,29088185,"2C1604,2C1603",1Y0604,KR_SERV01
4,29088182,"2C1602,BULK,2C1603",1Y0602,KR_SERV01
...,...,...,...,...
1032,18102926,2B1804,None,KR_SERV01
1033,19781401,2B1803,None,KR_SERV01
1034,29282854,2B1802,None,KR_SERV01
1035,29392877,2B1802,None,KR_SERV01


In [53]:
def get_insert_values(col_list):

    """
    cx_Oracle용 values 연속된 숫자 그룹 str만들때 사용 str 반환
    """
    list_len = len(col_list)

    value_list = []
    for i in range(1,list_len+1):
        value_list.append(":"+str(i))
    str_val = ','.join(value_list)
    
    return str_val

In [54]:
get_insert_values(df_bin.columns)

':1,:2,:3,:4'

In [77]:
# PROD_POSE

sub_list = list(df_stock['SUBINVENTORY'].unique())
sub_comment = list(map(lambda e : e.split('_')[1],sub_list))


['SERV01',
 'CSW',
 'SERV02',
 'PSJ',
 'JJH',
 'OPS',
 'CCK',
 'PJH',
 'DEMO',
 'LOCN01',
 'BSJ',
 'JKH',
 'CSS',
 'LJM',
 'SERV04',
 'OJK',
 'SERVBSA',
 'PGH',
 'CNH',
 'KS',
 'KGH',
 'SONGDO',
 'PJA',
 'BSH',
 'KHK',
 'KCW',
 'LSW',
 'KJS',
 'LTH',
 'KDS',
 'JGH',
 'KBC',
 'KTH',
 'LOCN03',
 'LOCN02']

In [92]:
df_prod_pose = pd.DataFrame([sub_list,sub_comment]).T
df_prod_pose

,0,1
0,KR_SERV01,SERV01
1,KR_CSW,CSW
2,KR_SERV02,SERV02
3,KR_PSJ,PSJ
4,KR_JJH,JJH
5,KR_OPS,OPS
6,KR_CCK,CCK
7,KR_PJH,PJH
8,KR_DEMO,DEMO
9,KR_LOCN01,LOCN01


In [94]:
df_prod_pose.rename(columns={0:'SUBINVENTROY',1:'COMMNET'},inplace=True)

In [171]:
cur = DataWarehouse()

In [172]:
table_name = 'PROD_POSE'
query = 'INSERT INTO '+ table_name + f' VALUES ({get_insert_values(df_prod_pose.columns)})'

df_tmp = df_prod_pose.values.tolist()
df_tmp_tuple =list(pd.Series(df_tmp).map(tuple))

query


cur.executemany(query,df_tmp_tuple,batcherrors= True)
cur.execute("commit")

df_prod_pose

,SUBINVENTROY,COMMNET
0,KR_SERV01,SERV01
1,KR_CSW,CSW
2,KR_SERV02,SERV02
3,KR_PSJ,PSJ
4,KR_JJH,JJH
5,KR_OPS,OPS
6,KR_CCK,CCK
7,KR_PJH,PJH
8,KR_DEMO,DEMO
9,KR_LOCN01,LOCN01


In [135]:
# PRODUCTS
pr_cols = ['ARTICLE_NUMBER',
'COUNTRY',
'PROD_CENTRE',
'PROD_GROUP',
'DESCRIPTION',
'PROD_STATUS_TYPE',
'BIN_CUR',
'BIN_OLD']
pr_cols

['ARTICLE_NUMBER',
 'COUNTRY',
 'PROD_CENTRE',
 'PROD_GROUP',
 'DESCRIPTION',
 'PROD_STATUS_TYPE',
 'BIN_CUR',
 'BIN_OLD']

In [120]:
df_bin

,ARTICLENUMBER,BIN(current),BIN(old),SUBINVENTORY
0,34-100390-002-9,"2C1804,2C1405",1Y0804,KR_SERV01
1,29128030,2B1801,1X0801,KR_SERV01
2,29089545,"BULK,2C1405",BULK,KR_SERV01
3,29088185,"2C1604,2C1603",1Y0604,KR_SERV01
4,29088182,"2C1602,BULK,2C1603",1Y0602,KR_SERV01
...,...,...,...,...
1032,18102926,2B1804,None,KR_SERV01
1033,19781401,2B1803,None,KR_SERV01
1034,29282854,2B1802,None,KR_SERV01
1035,29392877,2B1802,None,KR_SERV01


In [178]:
# join하기
df_stock_fin = df_stock[['ARTICLE NUMBER','COUNTRY','PROD CENTRE','PROD GROUP','DESCRIPTION','PROD STATUS TYPE']]
# df_stock
df_bin['ARTICLENUMBER'] = df_bin['ARTICLENUMBER'].astype('str')
# df_bin.merge(df_stock,)
df_merged = pd.merge(left=df_stock_fin,right=df_bin,left_on='ARTICLE NUMBER',right_on='ARTICLENUMBER',how='left')

In [179]:
# 중복제거
df_merged = df_merged.drop_duplicates(['ARTICLE NUMBER'])
df_merged.columns

Index(['ARTICLE NUMBER', 'COUNTRY', 'PROD CENTRE', 'PROD GROUP', 'DESCRIPTION',
       'PROD STATUS TYPE', 'ARTICLENUMBER', 'BIN(current)', 'BIN(old)',
       'SUBINVENTORY'],
      dtype='object')

In [210]:
pr_cols = pr_cols[:-1]

In [202]:
mapped_list = list(map(lambda e: e.replace('_',' '),pr_cols))
mapped_list = mapped_list[:-2]
mapped_list.append('BIN(current)')

In [204]:
df_merged[mapped_list]

,ARTICLE NUMBER,COUNTRY,PROD CENTRE,PROD GROUP,DESCRIPTION,PROD STATUS TYPE,BIN(current)
0,11000314,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,607 AKTA legacy parts,CU-950 KIT FOR AKTA,DstFGActv,2C1604
1,11000322,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,922 Bioprocess Downstream Parts - Standard,CU-950 MEMORY KIT,DstFGActv,2B1702
3,11000735,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,607 AKTA legacy parts,INTERFERENCE FILTER 280 NM,DstFGActv,2C1703
5,11000885,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,607 AKTA legacy parts,"PUMP SEAL P-901, INERT",DstFGActv,2B1802
12,11001080,GEAS_IN_LS_KR_KRS,48 RESEARCH SCIENCE SERVICE PARTS,868 Lab AKTA Classic Parts,DELIVERY ARM CPL.,DstFGActv,2C1602
...,...,...,...,...,...,...,...
1656,82000180,KRS,48 RESEARCH SCIENCE SERVICE PARTS,139 Computer Hardware,"Domestic purchasing computer set worth of 1,80...",DstFGActv,NaN
1657,82000250,KRS,48 RESEARCH SCIENCE SERVICE PARTS,139 Computer Hardware,"Domestic purchasing computer set worth of 2,40...",DstFGActv,NaN
1662,TBA_30001,KRS,20 CRYO CHAIN MANAGEMENT,157 VIA Thaw,VIA Thaw CB1000 Cryo-bag Thawing System,DstFGActv,NaN
1663,VFD_30006,KRS,20 CRYO CHAIN MANAGEMENT,156 VIA Freeze,VIA Freeze Duo Controlled Rate Freezer,DstFGActv,NaN


In [206]:
df_merged= df_merged[mapped_list]

In [211]:
df_merged.columns = pr_cols

In [213]:
df_prods = df_merged

In [220]:
df_prods = df_prods.fillna('None')


In [221]:
table_name = 'PRODUCTS'
query = 'INSERT INTO '+ table_name + f' VALUES ({get_insert_values(df_prods.columns)})'

df_tmp = df_prods.values.tolist()
df_tmp_tuple =list(pd.Series(df_tmp).map(tuple))


cur.executemany(query,df_tmp_tuple,batcherrors= True)
cur.execute("commit")

df_prods

,SUBINVENTROY,COMMNET
0,KR_SERV01,SERV01
1,KR_CSW,CSW
2,KR_SERV02,SERV02
3,KR_PSJ,PSJ
4,KR_JJH,JJH
5,KR_OPS,OPS
6,KR_CCK,CCK
7,KR_PJH,PJH
8,KR_DEMO,DEMO
9,KR_LOCN01,LOCN01


# SYSTEM_STOCK

In [223]:
system_stock_col =['SS_KEY', 'ARTICLE_NUMBER', 'SUBINVENTORY', 'LOCATION', 'QUANTITY', 'IN_DATE', 'EXPIRY_DATE', 'CURRENCY',
'LOT_COST', 'LOT_COST_IN_USD', 'STD_DAY']

In [236]:
df_st_cols = df_stock.columns
df_stock

,OU NAME,COUNTRY,PROD CENTRE,PROD GROUP,ARTICLE NUMBER,DESCRIPTION,PROD STATUS TYPE,LOT NUMBER,SERIAL NUMBER,SUBINVENTORY,LOCATION,QUANTITY,IN DATE,EXPIRY DATE,CURRENCY,LOT COST,LOT COST IN USD,LOT COST IN EUR
0,GEAS_OU_KRW_LS_KR,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,607 AKTA legacy parts,11000314,CU-950 KIT FOR AKTA,DstFGActv,None,None,KR_SERV01,None,1,2022-12-05,NaT,KRW,2291234,1817,1697
1,GEAS_OU_KRW_LS_KR,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,922 Bioprocess Downstream Parts - Standard,11000322,CU-950 MEMORY KIT,DstFGActv,None,None,KR_CSW,None,1,2023-01-05,NaT,KRW,175786,139,130
2,GEAS_OU_KRW_LS_KR,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,922 Bioprocess Downstream Parts - Standard,11000322,CU-950 MEMORY KIT,DstFGActv,None,None,KR_SERV01,None,1,2022-08-07,NaT,KRW,175786,139,130
3,GEAS_OU_KRW_LS_KR,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,607 AKTA legacy parts,11000735,INTERFERENCE FILTER 280 NM,DstFGActv,None,None,KR_SERV01,None,1,2022-12-12,NaT,KRW,420510,333,311
4,GEAS_OU_KRW_LS_KR,GEAS_IN_LS_KR_KRS,16 BIOPROCESS SERVICE PARTS,607 AKTA legacy parts,11000735,INTERFERENCE FILTER 280 NM,DstFGActv,None,None,KR_SERV01,None,1,2022-12-12,NaT,KRW,420510,333,311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,GEAS_OU_KRW_LS_KR,KRS,48 RESEARCH SCIENCE SERVICE PARTS,139 Computer Hardware,82000250,"Domestic purchasing computer set worth of 2,40...",DstFGActv,800673,800676,KR_LOCN02,None,1,2022-12-05,NaT,KRW,2500000,1982,1852
1661,GEAS_OU_KRW_LS_KR,KRS,48 RESEARCH SCIENCE SERVICE PARTS,139 Computer Hardware,82000250,"Domestic purchasing computer set worth of 2,40...",DstFGActv,800673,800677,KR_LOCN02,None,1,2022-12-05,NaT,KRW,2500000,1982,1852
1662,GEAS_OU_KRW_LS_KR,KRS,20 CRYO CHAIN MANAGEMENT,157 VIA Thaw,TBA_30001,VIA Thaw CB1000 Cryo-bag Thawing System,DstFGActv,None,TBA1118020,KR_DEMO,NULL.0.0.0.KR,1,2019-06-21,NaT,KRW,15036302,11924,11137
1663,GEAS_OU_KRW_LS_KR,KRS,20 CRYO CHAIN MANAGEMENT,156 VIA Freeze,VFD_30006,VIA Freeze Duo Controlled Rate Freezer,DstFGActv,None,VFD0920022,KR_DEMO,NULL.0.0.0.KR,1,2021-06-22,NaT,KRW,43671244,34631,32348


In [232]:
df_st_cols
need_cols = ['ARTICLE NUMBER', 'SUBINVENTORY', 'LOCATION', 'QUANTITY', 'IN DATE', 'EXPIRY DATE', 'CURRENCY', 'LOT COST','LOT COST IN USD']

In [238]:
df_stock_need = df_stock[need_cols]
df_stock_need.columns = system_stock_col[1:-1]
df_stock_need

,ARTICLE_NUMBER,SUBINVENTORY,LOCATION,QUANTITY,IN_DATE,EXPIRY_DATE,CURRENCY,LOT_COST,LOT_COST_IN_USD
0,11000314,KR_SERV01,None,1,2022-12-05,NaT,KRW,2291234,1817
1,11000322,KR_CSW,None,1,2023-01-05,NaT,KRW,175786,139
2,11000322,KR_SERV01,None,1,2022-08-07,NaT,KRW,175786,139
3,11000735,KR_SERV01,None,1,2022-12-12,NaT,KRW,420510,333
4,11000735,KR_SERV01,None,1,2022-12-12,NaT,KRW,420510,333
...,...,...,...,...,...,...,...,...,...
1660,82000250,KR_LOCN02,None,1,2022-12-05,NaT,KRW,2500000,1982
1661,82000250,KR_LOCN02,None,1,2022-12-05,NaT,KRW,2500000,1982
1662,TBA_30001,KR_DEMO,NULL.0.0.0.KR,1,2019-06-21,NaT,KRW,15036302,11924
1663,VFD_30006,KR_DEMO,NULL.0.0.0.KR,1,2021-06-22,NaT,KRW,43671244,34631


In [243]:
from datetime import datetime
now_date = str(datetime.now().date())

In [244]:
df_stock_need['STD_DAY'] = now_date

C:\Users\lms46\AppData\Local\Temp\ipykernel_32164\3175978381.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock_need['STD_DAY'] = now_date


In [246]:
df_stock_need['SS_KEY'] = None

C:\Users\lms46\AppData\Local\Temp\ipykernel_32164\4235258792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stock_need['SS_KEY'] = None


In [252]:
df_stock_need = df_stock_need[['SS_KEY', 'ARTICLE_NUMBER', 'SUBINVENTORY', 'LOCATION', 'QUANTITY', 'IN_DATE',
       'EXPIRY_DATE', 'CURRENCY', 'LOT_COST', 'LOT_COST_IN_USD', 'STD_DAY']]

In [253]:
table_name = 'SYSTEM_STOCK'
query = 'INSERT INTO '+ table_name + f' VALUES ({get_insert_values(df_stock_need.columns)})'

df_tmp = df_stock_need.values.tolist()
df_tmp_tuple =list(pd.Series(df_tmp).map(tuple))


cur.executemany(query,df_tmp_tuple,batcherrors= True)
cur.execute("commit")

In [ ]:
# 